# List train folders

##  Generate "train_folders.txt"

In [ ]:
!python list_train_folders.py -root_dir=/home/users/dawei.yang/data/source_eye_data/ -dir_type="a_open|glass_open|a_close|glass_close" > ../hf_work_20170805/file_list/train_folders.txt

# Disturb pictures

## Count number of all classes

In [ ]:
import os
import glob
import argparse

src_folders = '../hf_work_20170805/file_list/train_folders.txt'


def skip_comments_and_blank(file, cm='#'):
    lines = list()
    for line in file:
        line = line.strip()
        if not line.startswith(cm) and not line.isspace():
            lines.append(line)
    return lines


if __name__ == '__main__':
    dir_types = {}
    lines = skip_comments_and_blank(open(src_folders))
    for sub_dir in lines:
        dir_type = sub_dir.split(os.sep)[-1]
        if dir_type not in dir_types:
            print 'add', dir_type, 'in dir_types'
            dir_types[dir_type] = 0
       
        # count the number of files
        num = len(glob.glob(sub_dir+'/*'))
        dir_types[dir_type] += num
        print sub_dir, num
            
    print 'sum up:', dir_types

##  Generate disturb commands in disturb_pic.sh

In [ ]:
import os

PRE_PRINT_RES = True

# 需要处理的文件夹列表
src_folders = '../hf_work_20170805/file_list/train_folders.txt'
# 存储的脚本名称
fn_cmd = "disturb_in_pic.sh"
#save_path
save_path = '/home/users/dawei.yang/data/20170805_train_test'

# 扰动脚本
disturb_script = 'disturb_in_eye_pic.py'
# 扰动倍数
disturb_times = {'a_close':5, 'a_open':2, 'glass_close':13, 'glass_open':2,}



def skip_comments_and_blank(file, cm='#'):
    lines = list()
    for line in file:
        #print line
        line = line.strip()
        if not line.startswith(cm) and not line.isspace():
            lines.append(line)
    return lines


if __name__ == '__main__':
        
    out_f = open(fn_cmd, 'w')
    
    lines = skip_comments_and_blank(open(src_folders))
    for line in lines:
        if line[-1] == os.sep:
            line = line[:-1]
            
        class_dir = line.split(os.sep)[-1] # a_close
        disturb_time = disturb_times[class_dir]
        out_line = 'python ' + disturb_script
        out_line += ' -pic_dir=' + line + ' -save_path=' + save_path + ' -disturb_time=' + str(disturb_time)
            
        if PRE_PRINT_RES:
            print out_line
        out_f.write(out_line + '\n')
        
    out_f.truncate(out_f.tell()-1)
    out_f.close()

## Do disturb

`sh disturb_pic.sh`只能单线程运行，可以执行`python disturb_on_multi_cpus.py`，但是最好在terminal执行！！！

In [ ]:
!sh disturb_in_pic.sh

# list train files  for caffe

In [ ]:
!python list_train_files.py /home/users/dawei.yang/zhaoxu.li/glass_data/glass_20171111_train_test/train -recursive=1 > /home/users/dawei.yang/zhaoxu.li/mxnet_glass/datat/rain_files.txt
!python list_train_files.py /home/users/dawei.yang/zhaoxu.li/glass_data/glass_20171111_train_test/test -recursive=1 > /home/users/dawei.yang/zhaoxu.li/mxnet_glass/data/test_files.txt

## Convert to mxnet lst

In [ ]:
!python convert_caffeList_to_mxnetList.py ../data/train_files.txt ../data/ eye_train.lst
!python convert_caffeList_to_mxnetList.py ../data/train_files.txt ../data/eye_train.lst

## Get rec files for mxnet trainning

In [ ]:
!python im2rec.py --color=0 --encoding='.png' --num-thread=4 --reszie=32 ../data/1023_data/ ../data/1023_data/